In [1]:
import pandas as pd
import numpy as np
import ast

import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.abspath(''))))

from yumspeak_ml.params import *
from yumspeak_ml.ml_logic.data import clean_restaurant_data

In [2]:
len(CATS_TO_REMOVE)

338

In [3]:
df = pd.read_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/restaurant_data/all_restaurants_dataset.csv').drop(columns=['Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16'])
df.head()

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos
0,ChIJYR6HCJ4Z2jERChIQxjl2MhI,Red House Seafood Grand Copthorne,1110,Seafood restaurant,"['Seafood restaurant', 'Asian restaurant', 'Di...",4.2,"392 Havelock Rd, Level 2 Grand Copthorne Water...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipNp...
1,ChIJP2EDPqAZ2jERJKipSQq7qY8,Red House Seafood Nanyang (Clarke Quay),730,Seafood restaurant,['Seafood restaurant'],4.1,"3C River Valley Road 01-02/03, The Cannery, 17...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipMe...
2,ChIJSW7NnhAZ2jERj8p3W_nnhGE,Red House Seafood at Esplanade,186,Seafood restaurant,"['Seafood restaurant', 'Chinese restaurant']",4.4,"8 Raffles Ave., #01-14 / 16 Esplanade, Singapo...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipOJ...
3,ChIJNxLu-EAb2jER_a4Y4ahn8TU,Red Sparrow,496,Vietnamese restaurant,"['Vietnamese restaurant', 'Asian restaurant']",4.0,"11 Dempsey Rd, #01-18, 249673, Singapore 249673",https://www.google.com/maps/place/Red+Sparrow/...,['https://lh5.googleusercontent.com/p/AF1QipO8...
4,ChIJIds6rnUZ2jERPx6E0UCf6HM,Red Star Restaurant,3725,Cantonese restaurant,"['Cantonese restaurant', 'Chinese restaurant',...",3.7,"Block 54 Chin Swee Rd, #07-23, Singapore 160054",https://www.google.com/maps/place/Red+Star+Res...,['https://lh5.googleusercontent.com/p/AF1QipN-...


In [4]:
df = clean_restaurant_data(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4603 entries, 0 to 4602
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   place_id          4603 non-null   object 
 1   name              4603 non-null   object 
 2   reviews           4603 non-null   int64  
 3   main_category     4603 non-null   object 
 4   categories        4603 non-null   object 
 5   main_rating       4603 non-null   float64
 6   address           4603 non-null   object 
 7   link              4603 non-null   object 
 8   review_photos     4603 non-null   object 
 9   coordinates       4603 non-null   object 
 10  latitude          4603 non-null   float64
 11  longtitude        4603 non-null   float64
 12  full_postal_code  4603 non-null   object 
 13  postal_code       4603 non-null   object 
 14  district_code     4602 non-null   object 
 15  region            4602 non-null   object 
dtypes: float64(3), int64(1), object(12)
memory

In [9]:
#drop duplicates with subset 'place_id', 'name', 'reviews', 'address']
df = df.drop_duplicates(subset=['place_id', 'name', 'reviews', 'address'])

# remove irrelevant place based on main_categories
cats_to_remove = [cat.lower() for cat in CATS_TO_REMOVE]
mask = df['main_category'].str.lower().isin(cats_to_remove)
df = df[~mask]

# remove irrelevant place based by name
#df = df[~df['name'].isin(NAME_TO_DROP)]
df = df[~df['place_id'].isin(PLACE_ID_TO_DROP)]

# fill na in main_category and categories(as ['unknown'])
df['main_category'].fillna('unknown', inplace=True)
df['main_category'] = df['main_category'].apply(lambda x: x.replace(' restaurant', ''))
df['categories'] = df['categories'].fillna("['unknown']").apply(ast.literal_eval)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4797 entries, 0 to 10093
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       4797 non-null   object 
 1   name           4797 non-null   object 
 2   reviews        4797 non-null   int64  
 3   main_category  4797 non-null   object 
 4   categories     4797 non-null   object 
 5   main_rating    4797 non-null   float64
 6   address        4771 non-null   object 
 7   link           4797 non-null   object 
 8   review_photos  4797 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 374.8+ KB


In [6]:
df.to_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/restaurant_data/final_restaurant_b1.csv', index=False, header=True)

In [19]:
contains_pte_ltd = df['name'].str.contains('Pte Ltd', case=False, na=False)
contains_pte_ltd.sum()

52

In [20]:
unknown_main_cat = df['main_category'] == 'unknown'
unknown_main_cat.sum()

40

In [21]:
no_rating = df['main_rating'] == 0
no_rating.sum()

215

In [22]:
restaurant = df['main_category'].str.contains('restaurant', case=False, na=False)
restaurant.sum()

4150

In [23]:
to_remove = df[(~restaurant & contains_pte_ltd & no_rating) | unknown_main_cat].reset_index(drop=True)
to_remove.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       40 non-null     object 
 1   name           40 non-null     object 
 2   reviews        40 non-null     int64  
 3   main_category  40 non-null     object 
 4   categories     40 non-null     object 
 5   main_rating    40 non-null     float64
 6   address        38 non-null     object 
 7   link           40 non-null     object 
 8   review_photos  40 non-null     object 
dtypes: float64(1), int64(1), object(7)
memory usage: 2.9+ KB


In [24]:
to_remove

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos
0,ChIJg0UVB4IZ2jERfWKNEG8EyDc,Smile Dessert,0,unknown,[unknown],0.0,"717 Havelock Rd, Singapore 160022",https://www.google.com/maps/place/Smile+Desser...,[]
1,ChIJScNPoUKjAmAR3OO48IT78tQ,Takayama,0,unknown,[unknown],0.0,"Gifu, Japan","https://www.google.com/maps/place/Takayama,+Gi...",[]
2,ChIJc-gZD5EZ2jERRuJ5h8wWLH4,Thaiexpress Concepts Cineleisure Orchard,1,unknown,[unknown],3.0,"8 Grange Rd, Cineleisure Orchard, Singapore 23...",https://www.google.com/maps/place/Thaiexpress+...,[]
3,ChIJibiOH5kZ2jERcW3MZYgn_RI,The Sushi Bar Dining,1,unknown,[unknown],1.0,"280 River Valley Rd, Singapore 238321",https://www.google.com/maps/place/The+Sushi+Ba...,[]
4,ChIJyWPGSAIZ2jERw7arCVkYYdY,Vietnamese Cuisine,3,unknown,[unknown],4.7,"7 Hoe Chiang Rd, Singapore 089313",https://www.google.com/maps/place/Vietnamese+C...,['https://lh5.googleusercontent.com/p/AF1QipOe...
5,ChIJNcn6xtYb2jERKTlNIZRmQis,Ma La Xiang Guo,0,unknown,[unknown],0.0,"164 Bukit Merah Central, Singapore 150164",https://www.google.com/maps/place/Ma+La+Xiang+...,[]
6,ChIJPTjULjF0K4cRDfdv8pqL2ec,Rise,0,unknown,[unknown],0.0,"Scottsdale, AZ 85260, United States",https://www.google.com/maps/place/Rise/data=!4...,[]
7,ChIJrwRa3javK4cRKsgBw7yd65c,Rise,0,unknown,[unknown],0.0,"Mesa, AZ 85206, United States",https://www.google.com/maps/place/Rise/data=!4...,[]
8,ChIJ1239LW0Z2jERjAM826R9LvI,Amò,1,unknown,[unknown],4.0,"57 Tras St, Singapore 078996",https://www.google.com/maps/place/Am%C3%B2/dat...,[]
9,ChIJa1-WpnIZ2jERHQAr226M9eo,Amò,0,unknown,[unknown],0.0,"18 Tg Pagar Rd, Singapore 088441",https://www.google.com/maps/place/Am%C3%B2/dat...,[]


In [25]:
df['main_category'] = df['main_category'].apply(lambda x: x.replace(' restaurant', ''))

In [11]:
df.main_category.unique()

array(['Seafood', 'Vietnamese', 'Cantonese', 'Indian', 'Restaurant',
       'Dim sum', 'Cafe', 'Singaporean', 'Indonesian', 'Halal',
       'Fine dining', 'Modern European', 'Asian fusion', 'Thai', 'Nyonya',
       'Chinese', 'French', 'Hot pot', 'Italian', 'Vegetarian',
       'Mediterranean', 'Meat dish', 'Pizza', 'Family', 'Korean',
       'Japanese', 'Grill', 'Steak house', 'Sushi', 'Hamburger', 'Asian',
       'Bar', 'Traditional', 'Tapas', 'Mexican', 'Chinese noodle',
       'Pakistani', 'Sichuan', 'Turkish', 'Modern French', 'Salad shop',
       'Seafood donburi', 'Tex-Mex', 'Chicken', 'South Indian', 'Andhra',
       'Lebanese', 'Fish and seafood', 'Izakaya', 'Authentic Japanese',
       'Modern izakaya', 'Conveyor belt sushi', 'Unagi', 'Oyster bar',
       'Bistro', 'Steamboat', 'Indian Muslim Restaurant', 'Malaysian',
       'Nasi', 'Brunch', 'Bakso', 'Breakfast', 'Fried chicken takeaway',
       'Padang', 'Ramen', 'Cha chaan teng (Hong Kong-style cafe)',
       'Tempura donb

In [12]:
df

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos
0,ChIJYR6HCJ4Z2jERChIQxjl2MhI,Red House Seafood Grand Copthorne,1110,Seafood,"[Seafood restaurant, Asian restaurant, Dim sum...",4.2,"392 Havelock Rd, Level 2 Grand Copthorne Water...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipNp...
1,ChIJP2EDPqAZ2jERJKipSQq7qY8,Red House Seafood Nanyang (Clarke Quay),730,Seafood,[Seafood restaurant],4.1,"3C River Valley Road 01-02/03, The Cannery, 17...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipMe...
2,ChIJSW7NnhAZ2jERj8p3W_nnhGE,Red House Seafood at Esplanade,186,Seafood,"[Seafood restaurant, Chinese restaurant]",4.4,"8 Raffles Ave., #01-14 / 16 Esplanade, Singapo...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipOJ...
3,ChIJNxLu-EAb2jER_a4Y4ahn8TU,Red Sparrow,496,Vietnamese,"[Vietnamese restaurant, Asian restaurant]",4.0,"11 Dempsey Rd, #01-18, 249673, Singapore 249673",https://www.google.com/maps/place/Red+Sparrow/...,['https://lh5.googleusercontent.com/p/AF1QipO8...
4,ChIJIds6rnUZ2jERPx6E0UCf6HM,Red Star Restaurant,3725,Cantonese,"[Cantonese restaurant, Chinese restaurant, Dim...",3.7,"Block 54 Chin Swee Rd, #07-23, Singapore 160054",https://www.google.com/maps/place/Red+Star+Res...,['https://lh5.googleusercontent.com/p/AF1QipN-...
...,...,...,...,...,...,...,...,...,...
10089,ChIJA2UPtIwZ2jERE6WwKP0x278,Hua Ting Steamboat,70,Steamboat,"[Steamboat restaurant, Chinese restaurant]",4.2,"Mezzanine level #01-08, Claymore Connect, 442 ...",https://www.google.com/maps/place/Hua+Ting+Ste...,['https://lh5.googleusercontent.com/p/AF1QipM4...
10090,ChIJ3XkDF-EW2jERnoxsx_i5CcY,Hualong Fishhead Steamboat,173,Steamboat,"[Steamboat restaurant, Asian restaurant, Chine...",3.8,"347 Ang Mo Kio Ave 3, #01-2142 Blk 347, Singap...",https://www.google.com/maps/place/Hualong+Fish...,['https://lh5.googleusercontent.com/p/AF1QipMY...
10091,ChIJLfNb-H0X2jERHknheYoXfgY,Hualong Fishhead Steamboat,45,Chinese,[Chinese restaurant],3.8,"978 Toa Payoh N, #02-01, Singapore 319001",https://www.google.com/maps/place/Hualong+Fish...,['https://lh5.googleusercontent.com/p/AF1QipMl...
10092,ChIJ6-YzAysX2jERKmeXREbO7VM,Hualong Fishhead Steamboat,8,Chinese,[Chinese restaurant],2.6,"Blk 631 Hougang Ave 8, #01-06, Singapore 530631",https://www.google.com/maps/place/Hualong+Fish...,['https://lh5.googleusercontent.com/p/AF1QipOW...
